In [1]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import re
import sys

In [2]:
# helper functions
def loadJson(path):
    '''returns datarfame from json file'''
    with open(path,'r') as fp: #load historic tweets
        tweets_json = json.load(fp)
    return pd.read_json(tweets_json)
def saveText(dictionary,path):
    '''save texts stored as values of a dictionary with their keys as file name'''
    for category in dictionary:
        with open(path+category+'.txt', "w",encoding="utf-8") as text_file:
            text_file.write(dictionary[category])
class progress_bar(object):
    def __init__(self,length,startingValue = 0):
        self.length = length
        self.progress = startingValue
    def increase(self,by = 1):
        self.progress += 1/self.length
        self.update_progress(self.progress)
    def update_progress(self,progress):
        barLength = 10 # Modify this to change the length of the progress bar
        status = ""
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Halt...\r\n"
        if progress >= 1:
            progress = 1
            status = "Done...\r\n"
        block = int(round(barLength*progress))
        text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), round(progress*100,2), status)
        sys.stdout.write(text)
        sys.stdout.flush()

In [3]:
#text extraction functions
def getText(l):
    if not(type(l)==list):
        return ''
    t = ''
    for e in l:
        t += e['text'].split('http')[0]
    return t
    #return re.sub(r'(\s)@\w+', r'\1', t) #if we finally remove mentions
def getCategoryText(l):
    catText = ''
    for e in l:
        catText += getText(e)
        catText += '\n'
    return catText

In [4]:
#load accounts
accounts = pd.read_csv('../data collection/Data/accounts.csv',sep=';')
accounts.loc[accounts.Category == 'Funny/Entertainment','Category'] = 'Funny&Entertainment' #/ is problematic when saving

In [5]:
#initialize results
categoryTexts = {cat:'' for cat in accounts.Category.unique()}

In [6]:
load_from = '../data collection/Data/ModelData/Tweets'
pathlist = list(Path(load_from).glob('*-04-20.json'))
pb = progress_bar(len(pathlist))
for path in pathlist:
    date = path.name[13:-5]
    tweets = loadJson(path)
    merged = pd.merge(accounts,tweets,on='Account')
    for cat in categoryTexts:
        categoryTexts[cat] += getCategoryText(merged[merged.Category == cat][date])
    pb.increase()

Percent: [##########] 100% Done...


In [7]:
save_to = 'Texts/'
saveText(categoryTexts, save_to)